In [4]:
import pandas as pd

# Fájl elérési útja
file_path = r'C:\\Users\\Felhasználó\\Desktop\\PetersonBarney\\verified_pb.data'

# Adatok beolvasása egy pandas DataFrame-be
column_names = ['gender', 'speaker', 'phoneme', 'phoneme_in_ascii', 'F0', 'F1', 'F2', 'F3']
data = pd.read_csv(file_path, delim_whitespace=True, names=column_names)

print(data.head())

   gender  speaker  phoneme phoneme_in_ascii     F0     F1      F2      F3
0       1        1        1               IY  160.0  240.0  2280.0  2850.0
1       1        1        1               IY  186.0  280.0  2400.0  2790.0
2       1        1        2               IH  203.0  390.0  2030.0  2640.0
3       1        1        2               IH  192.0  310.0  1980.0  2550.0
4       1        1        3               EH  161.0  490.0  1870.0  2420.0


C:\Users\Felhasználó\AppData\Local\Temp\ipykernel_16072\4101704356.py:8: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(file_path, delim_whitespace=True, names=column_names)


# Adatok felosztása
Először felosztjuk az adatokat tanuló, validációs és teszt halmazra alkalmazva az óran elhangzott 80% - 10% - 10% arányt.

In [21]:
from sklearn.model_selection import train_test_split

# Beolvassuk az adatokat a slideban megadott oszlopnevekkel
column_names = ['gender', 'speaker', 'phoneme', 'phoneme_in_ascii', 'F0', 'F1', 'F2', 'F3']
data = pd.read_csv(r'C:\\Users\\Felhasználó\\Desktop\\PetersonBarney\\verified_pb.data', delim_whitespace=True, names=column_names)

# Bemeneti adatok és címke kiválasztása
X = data[['F0', 'F1', 'F2', 'F3']]  # Az akusztikai frekvenciák
y = data['phoneme']  # A fonéma, amit osztályozni fogunk

# Adatok felosztása tanuló (80%), validációs (10%) és teszt adatokra (10%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Ellenőrzés, hogy helyesen osztottuk-e fel az adatokat
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")


Training set size: 1216
Validation set size: 152
Test set size: 152


C:\Users\Felhasználó\AppData\Local\Temp\ipykernel_16072\1643603563.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(r'C:\\Users\\Felhasználó\\Desktop\\PetersonBarney\\verified_pb.data', delim_whitespace=True, names=column_names)


# Egy baseline modell létrehozása
Itt egy Gauss-keverék modellt tanítunk az adaton.

In [22]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import accuracy_score

# GMM modell létrehozása
gmm = GaussianMixture(n_components=len(y.unique()), covariance_type='full', random_state=42)

# A modell betanítása a tréning adatokon
gmm.fit(X_train)

# A modell kiértékelése a validációs adatokon
y_pred_gmm = gmm.predict(X_val)
accuracy_gmm = accuracy_score(y_val, y_pred_gmm)

print(f"Validation accuracy of GMM baseline model: {accuracy_gmm * 100:.2f}%")


Validation accuracy of GMM baseline model: 15.79%


# Egy másik modell az összehasonlításhoz
Most egy egyszerű K-NN osztályozót tanítunk az adaton.

In [23]:
from sklearn.neighbors import KNeighborsClassifier

# K-NN modell létrehozása (k=3)
knn = KNeighborsClassifier(n_neighbors=3)

# A modell betanítása a tréning adatokon
knn.fit(X_train, y_train)

# A modell kiértékelése a validációs adatokon
y_pred_knn = knn.predict(X_val)
accuracy_knn = accuracy_score(y_val, y_pred_knn)

print(f"Validation accuracy of K-NN model: {accuracy_knn * 100:.2f}%")


Validation accuracy of K-NN model: 84.21%


# Kiértékeljük mind a kettőt a teszt adaton

In [24]:
# Tesztelés a GMM baseline modellel
y_pred_test_gmm = gmm.predict(X_test)
accuracy_test_gmm = accuracy_score(y_test, y_pred_test_gmm)

# Tesztelés a K-NN modellel
y_pred_test_knn = knn.predict(X_test)
accuracy_test_knn = accuracy_score(y_test, y_pred_test_knn)

print(f"Test accuracy of GMM baseline model: {accuracy_test_gmm * 100:.2f}%")
print(f"Test accuracy of K-NN model: {accuracy_test_knn * 100:.2f}%")


Test accuracy of GMM baseline model: 18.42%
Test accuracy of K-NN model: 84.87%


# Értékelés:

* Láthatóan a Gauss-keverék modell sokkal gyengébben teljesít, mint az egyszerű K-NN algoritmus.
* Ennek több oka is lehet például, hogy nincs elég adatunk a feladat megoldásához.
* Vagy mindössze a GMM a fonémák valószínűségi eloszlásait próbálja modellezni, de ezek az adatok valószínűleg nem követik jól a Gauss-eloszlásokat, ezért nem illeszkedik megfelelően. Így a K-NN, ami az adatpontok közelségét használja az osztályozáshoz jobban tud teljesíteni.